In [1]:
using BenchmarkTools
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

# versione INIZIALE di circle()

In [2]:
function circle(radius=1., angle=2*pi)
    function circle0(shape=[36])     
        V, EV = cuboidGrid(shape)
        V = (angle/shape[1])*V
        V = hcat(map(u->[radius*cos(u); radius*sin(u)], V)...)
        W, EW = simplifyCells(V, EV)
        return W, EW
    end
    return circle0
end

circle (generic function with 3 methods)

In [3]:
 @btime D,Z= Lar.circle()()

  140.685 μs (1838 allocations: 139.64 KiB)


([1.0 0.98481 … 0.93969 0.98481; 0.0 0.17365 … -0.34202 -0.17365], [[2, 1], [2, 3], [4, 3], [4, 5], [5, 6], [7, 6], [7, 8], [9, 8], [9, 10], [10, 11]  …  [27, 28], [29, 28], [29, 30], [31, 30], [31, 32], [32, 33], [34, 33], [35, 34], [35, 36], [36, 1]])

In [5]:
@benchmark Lar.circle()()

BenchmarkTools.Trial: 
  memory estimate:  139.58 KiB
  allocs estimate:  1836
  --------------
  minimum time:     141.559 μs (0.00% GC)
  median time:      145.319 μs (0.00% GC)
  mean time:        160.438 μs (7.89% GC)
  maximum time:     4.798 ms (96.03% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [4]:
@code_warntype circle()()

Variables
  #self#::var"#circle0#2"{Float64,Float64}

Body::Tuple{Any,Any}
1 ─ %1 = Base.vect(36)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Any,Any}
└──      return %2


# versione MODIFICATA di circle()

In [7]:
function circle(radius=1., angle=2*pi)
    function circle0(shape=[36]) 
        V = [zero(eltype{Int64,1})]  
        EV = [zero(eltype{Int64,1})] 
        V, EV = cuboidGrid(shape)
        V = (angle/shape[1])*V
        V = hcat(map(u->[radius*cos(u); radius*sin(u)], V)...)
        W, EW = simplifyCells(V, EV)
        return W, EW
    end
    return circle0
end 

circle (generic function with 3 methods)

In [8]:
 @btime D,Z= Lar.circle()()

  140.601 μs (1838 allocations: 139.64 KiB)


([1.0 0.98481 … 0.93969 0.98481; 0.0 0.17365 … -0.34202 -0.17365], [[2, 1], [2, 3], [4, 3], [4, 5], [5, 6], [7, 6], [7, 8], [9, 8], [9, 10], [10, 11]  …  [27, 28], [29, 28], [29, 30], [31, 30], [31, 32], [32, 33], [34, 33], [35, 34], [35, 36], [36, 1]])

In [9]:
@benchmark Lar.circle()()

BenchmarkTools.Trial: 
  memory estimate:  139.58 KiB
  allocs estimate:  1836
  --------------
  minimum time:     136.841 μs (0.00% GC)
  median time:      145.526 μs (0.00% GC)
  mean time:        162.181 μs (7.99% GC)
  maximum time:     4.815 ms (95.86% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [8]:
@code_warntype circle()()

Variables
  #self#::var"#circle0#7"{Float64,Float64}

Body::Union{}
1 ─ %1 = Base.vect(36)::Array{Int64,1}
│        (#self#)(%1)
└──      Core.Compiler.Const(:(return %2), false)


La *nuova versione* non migliora i tempi di esecuzione della funzione iniziale ma rende la variabile V tipizzata . Quindi la rende più stabile .